## 1. Multi-Threading:
Multi-threading is a technique used in programming to achieve concurrent execution of multiple threads within a single process. In Python, you can utilize multi-threading to perform multiple tasks simultaneously and improve the overall performance of your program.

Python provides a built-in module called threading for multi-threading.

A thread is an entity within a process that can be scheduled for execution. Also, it is the smallest unit of processing that can be performed in an OS (Operating System). In simple words, a thread is a sequence of such instructions within a program that can be executed independently of other code. For simplicity, you can assume that a thread is simply a subset of a process! A thread contains all this information in a Thread Control Block (TCB):

#### Thread Identifier: Unique id (TID) is assigned to every new thread

#### Stack pointer: Points to the thread’s stack in the process. The stack contains the local variables under the thread’s scope.

#### Program counter: a register that stores the address of the instruction currently being executed by a thread.

##### Thread state: can be running, ready, waiting, starting, or done.

##### Thread’s register set: registers assigned to thread for computations.

##### Parent process Pointer: A pointer to the Process control block (PCB) of the process that the thread lives on.

Multiple threads can exist within one process where:

Each thread contains its own register set and local variables (stored in the stack).

All threads of a process share global variables (stored in heap) and the program code.

In [42]:
import threading
import time

def print_cube(num):
    # function to print cube of given num
    print("Cube: {}".format(num * num * num))
    time.sleep(2)  # Simulate some time-consuming task


def print_square(num):
    # function to print square of given num
    print("Square: {}".format(num * num))
    time.sleep(1)  # Simulate some time-consuming task


if __name__ == "__main__":
    # creating threads
    t1 = threading.Thread(target=print_square, args=(20,))  # args is value passed to function
    t2 = threading.Thread(target=print_cube, args=(10,))

    # starting thread 1
    start_time = time.time()
    t1.start()

    # starting thread 2
    t2.start()

    # wait until both threads are completely executed
    t1.join()
    t2.join()

    # calculate and print the time taken by each thread
    t1_time = time.time() - start_time
    print("Thread 1 execution time:", t1_time, "seconds")

    t2_time = time.time() - start_time
    print("Thread 2 execution time:", t2_time, "seconds")
    

    # both threads completely executed
print("Done!")


Square: 400
Cube: 1000
Thread 1 execution time: 2.0042378902435303 seconds
Thread 2 execution time: 2.0042378902435303 seconds
Done!


#### In this example,
we use os.getpid() function to get the ID of the current process. We use threading.main_thread() function to get the main thread object.

In [46]:
# Python program to illustrate the concept
# of threading
import threading
import os
import time


def task1():
	print("Task 1 assigned to thread: {}".format(threading.current_thread().name))
	print("ID of process running task 1: {}".format(os.getpid()))

def task2():
	print("Task 2 assigned to thread: {}".format(threading.current_thread().name))
	print("ID of process running task 2: {}".format(os.getpid()))

if __name__ == "__main__":

	# print ID of current process
	print("ID of process running main program: {}".format(os.getpid()))

	# print name of main thread
	print("Main thread name: {}".format(threading.current_thread().name))

	# creating threads
	t1 = threading.Thread(target=task1, name='t1')
	t2 = threading.Thread(target=task2, name='t2')

	# starting threads
	t1.start()
	t2.start()

	# wait until all threads finish
	t1.join()
	t2.join()


ID of process running main program: 13896
Main thread name: MainThread
Task 1 assigned to thread: t1
ID of process running task 1: 13896
Task 2 assigned to thread: t2
ID of process running task 2: 13896


## Python ThreadPool

In [47]:
import concurrent.futures

def worker():
	print("Worker thread running")

# create a thread pool with 2 threads
pool = concurrent.futures.ThreadPoolExecutor(max_workers=2)

# submit tasks to the pool
pool.submit(worker)
pool.submit(worker)

# wait for all tasks to complete
pool.shutdown(wait=True)

print("Main thread continuing to run")


Worker thread running
Worker thread running
Main thread continuing to run


we define a function worker that will run in a thread. We create a ThreadPoolExecutor with a maximum of 2 worker threads. We then submit two tasks to the pool using the submit method. The pool manages the execution of the tasks in its worker threads. We use the shutdown method to wait for all tasks to complete before the main thread continues.

## Thread synchronization:

#### concept of race condition

In [51]:
import threading

# global variable x
x = 0

def increment():
	"""
	function to increment global variable x
	"""
	global x
	x += 1

def thread_task():
	"""
	task for thread
	calls increment function 100000 times.
	"""
	for _ in range(100000):
		increment()

def main_task():
	global x
	# setting global variable x as 0
	x = 0

	# creating threads
	t1 = threading.Thread(target=thread_task)
	t2 = threading.Thread(target=thread_task)

	# start threads
	t1.start()
	t2.start()

	# wait until threads finish their job
	t1.join()
	t2.join()

if __name__ == "__main__":
	for i in range(10):
		main_task()
		print("Iteration {0}: x = {1}".format(i,x))


Iteration 0: x = 200000
Iteration 1: x = 200000
Iteration 2: x = 200000
Iteration 3: x = 200000
Iteration 4: x = 200000
Iteration 5: x = 200000
Iteration 6: x = 200000
Iteration 7: x = 200000
Iteration 8: x = 200000
Iteration 9: x = 200000


#### In above program:

Two threads t1 and t2 are created in main_task function and global variable x is set to 0.

Each thread has a target function thread_task in which increment function is called 100000 times.

increment function will increment the global variable x by 1 in each call.

The expected final value of x is 200000 but what we get in 10 iterations of main_task function is some different values.


This happens due to concurrent access of threads to the shared variable x. This unpredictability in value of x is nothing but race condition.

## Using Locks:
threading module provides a Lock class to deal with the race conditions.

In [50]:
import threading

# global variable x
x = 0

def increment():
	"""
	function to increment global variable x
	"""
	global x
	x += 1

def thread_task(lock):
	"""
	task for thread
	calls increment function 100000 times.
	"""
	for _ in range(100000):
		lock.acquire()
		increment()
		lock.release()

def main_task():
	global x
	# setting global variable x as 0
	x = 0

	# creating a lock
	lock = threading.Lock()

	# creating threads
	t1 = threading.Thread(target=thread_task, args=(lock,))
	t2 = threading.Thread(target=thread_task, args=(lock,))

	# start threads
	t1.start()
	t2.start()

	# wait until threads finish their job
	t1.join()
	t2.join()

if __name__ == "__main__":
	for i in range(10):
		main_task()
		print("Iteration {0}: x = {1}".format(i,x))


Iteration 0: x = 200000
Iteration 1: x = 200000
Iteration 2: x = 200000
Iteration 3: x = 200000
Iteration 4: x = 200000
Iteration 5: x = 200000
Iteration 6: x = 200000
Iteration 7: x = 200000
Iteration 8: x = 200000
Iteration 9: x = 200000
